In [2]:
# RNN(Long Short-Term Memory, LSTM)

In [80]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [81]:
def mergeData():
    SaveFile_Name = 'data.csv'
    file_list = os.listdir('data')
    df = pd.read_csv('data'+'\\'+file_list[0])
    df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv('data'+'\\'+file_list[i])
        df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+')

In [82]:
def readData():
    train = pd.read_csv("data.csv")
    return train

In [83]:
def changeYear(data):
    for i in range(0,data.shape[0]):
        Date=data["日期"][i].split('/')
        year,month,date=Date[0],Date[1],Date[2]
        year=str(int(year)+1911)
        data.loc[i,"日期"]=year+'/'+month+'/'+date
    return data

In [84]:
# Augment Features
def augFeatures(data):
  data["日期"] = pd.to_datetime(data["日期"])
  data["年"] = data["日期"].dt.year
  data["月"] = data["日期"].dt.month
  data["日"] = data["日期"].dt.day
  data["第幾日"] = data["日期"].dt.dayofweek
  return data

In [85]:
def manage(data):
    for i in range(0,data.shape[0]):
        if data["漲跌價差"][i]=='X0.00':
            data.loc[i,"漲跌價差"]=str(int(data["收盤價"][i])-int(data["收盤價"][i-1]))
    data=data.drop(["日期"], axis=1)
    data=data.drop(["成交股數"], axis=1)
    data=data.drop(["成交金額"], axis=1)
    data=data.drop(["漲跌價差"], axis=1)
    data=data.drop(["成交筆數"], axis=1)
    data=data.convert_objects(convert_numeric=True)
    return data

In [86]:
def normalize(data):
    datanormalize=data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return datanormalize

In [112]:
def buildTrain(train, pastDay=1, futureDay=1):
    X_train, Y_train, Z_train= [], [], []
    X,Y,Z=[],[],[]
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["開盤價"]))
        Z_train.append(np.array(train.iloc[i:i+pastDay]["開盤價"]))
    X=np.array(X_train)
    Y=np.array(Y_train)
    Z=np.array(Z_train)
    Y=(Y-Z)/2
    Y_train=[]
    for i in range(len(Y)):
        if Y[i]<-5:
            Y_train.append(np.array([1,0,0,0,0,0,0,0,0,0,0,0]))
        elif -5<=Y[i]<-4:
            Y_train.append(np.array([0,1,0,0,0,0,0,0,0,0,0,0]))
        elif -4<=Y[i]<-3:
            Y_train.append(np.array([0,0,1,0,0,0,0,0,0,0,0,0]))
        elif -3<=Y[i]<-2:
            Y_train.append(np.array([0,0,0,1,0,0,0,0,0,0,0,0]))
        elif -2<=Y[i]<-1:
            Y_train.append(np.array([0,0,0,0,1,0,0,0,0,0,0,0]))
        elif -1<=Y[i]<0:
            Y_train.append(np.array([0,0,0,0,0,1,0,0,0,0,0,0]))
        elif 0<=Y[i]<1:
            Y_train.append(np.array([0,0,0,0,0,0,1,0,0,0,0,0]))
        elif 1<=Y[i]<2:
            Y_train.append(np.array([0,0,0,0,0,0,0,1,0,0,0,0]))
        elif 2<=Y[i]<3:
            Y_train.append(np.array([0,0,0,0,0,0,0,0,1,0,0,0]))
        elif 3<=Y[i]<4:
            Y_train.append(np.array([0,0,0,0,0,0,0,0,0,1,0,0]))
        elif 4<=Y[i]<5:
            Y_train.append(np.array([0,0,0,0,0,0,0,0,0,0,1,0]))
        elif 5<=Y[i]:
            Y_train.append(np.array([0,0,0,0,0,0,0,0,0,0,0,1]))
    Y=np.array(Y_train)
    return X, Y

In [113]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [114]:
# 將Training Data取一部份當作Validation Data
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [119]:
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(12)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [120]:

mergeData()
train=readData()
train=changeYear(train)
train=augFeatures(train)
train=manage(train)
temp=train
train=normalize(train)
train_x1, train_y1 = buildTrain(train, 1, 1)
train_x2, train_y2 = buildTrain(temp, 1, 1)
train_x, train_y = train_x1,train_y2
train_x, train_y = shuffle(train_x, train_y )
# split training data and validation data
train_x, train_y , val_x, val_y = splitData(train_x, train_y , 0.1)
train_y = train_y[:,np.newaxis]
val_y = val_y[:,np.newaxis]

model = buildOneToOneModel(train_x.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")

model.fit(train_x, train_y, epochs=1000, batch_size=50, validation_data=(val_x, val_y), callbacks=[callback])


C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 8), return_sequences=True)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 1, 10)             760       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 1, 12)             132       
Total params: 892
Trainable params: 892
Non-trainable params: 0
_________________________________________________________________
Train on 5571 samples, validate on 618 samples
Epoch 1/1000
5571/5571 [==============================] - 2s 281us/step - loss: 0.0749 - val_loss: 0.0665
Epoch 2/1000
5571/5571 [==============================] - 0s 32us/step - loss: 0.0611 - val_loss: 0.0582
Epoch 3/1000
5571/5571 [==============================] - 0s 57us/step - loss: 0.0573 - val_loss: 0.0577
Epoch 4/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0571 - val_loss: 0.0577
Epoch 5/1000
5571/5571 [==============================] - 0s 32us/step -

5571/5571 [==============================] - 0s 31us/step - loss: 0.0567 - val_loss: 0.0576
Epoch 73/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0567 - val_loss: 0.0576
Epoch 74/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0567 - val_loss: 0.0575
Epoch 75/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0567 - val_loss: 0.0576
Epoch 76/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0567 - val_loss: 0.0576
Epoch 77/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0567 - val_loss: 0.0575
Epoch 78/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0567 - val_loss: 0.0576
Epoch 79/1000
5571/5571 [==============================] - 0s 28us/step - loss: 0.0567 - val_loss: 0.0575
Epoch 80/1000
5571/5571 [==============================] - 0s 28us/step - loss: 0.0567 - val_loss: 0.0575
Epoch 81/1000
5571/5571 [==============================] - 0

Epoch 149/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0563 - val_loss: 0.0576
Epoch 150/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0563 - val_loss: 0.0576
Epoch 151/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0563 - val_loss: 0.0575
Epoch 152/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0563 - val_loss: 0.0576
Epoch 153/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0563 - val_loss: 0.0576
Epoch 154/1000
5571/5571 [==============================] - 0s 32us/step - loss: 0.0563 - val_loss: 0.0576
Epoch 155/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0563 - val_loss: 0.0577
Epoch 156/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0562 - val_loss: 0.0576
Epoch 157/1000
5571/5571 [==============================] - 0s 28us/step - loss: 0.0562 - val_loss: 0.0575
Epoch 158/1000
5571/5571 [===========

5571/5571 [==============================] - 0s 33us/step - loss: 0.0556 - val_loss: 0.0572
Epoch 226/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0556 - val_loss: 0.0572
Epoch 227/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0556 - val_loss: 0.0573
Epoch 228/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0556 - val_loss: 0.0572
Epoch 229/1000
5571/5571 [==============================] - 0s 32us/step - loss: 0.0555 - val_loss: 0.0570
Epoch 230/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0555 - val_loss: 0.0571
Epoch 231/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0555 - val_loss: 0.0570
Epoch 232/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0554 - val_loss: 0.0570
Epoch 233/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0555 - val_loss: 0.0572
Epoch 234/1000
5571/5571 [==========================

5571/5571 [==============================] - 0s 36us/step - loss: 0.0537 - val_loss: 0.0554
Epoch 302/1000
5571/5571 [==============================] - 0s 35us/step - loss: 0.0537 - val_loss: 0.0553
Epoch 303/1000
5571/5571 [==============================] - 0s 34us/step - loss: 0.0537 - val_loss: 0.0552
Epoch 304/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0536 - val_loss: 0.0553
Epoch 305/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0536 - val_loss: 0.0552
Epoch 306/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0536 - val_loss: 0.0551
Epoch 307/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0536 - val_loss: 0.0554
Epoch 308/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0536 - val_loss: 0.0551
Epoch 309/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0535 - val_loss: 0.0551
Epoch 310/1000
5571/5571 [==========================

5571/5571 [==============================] - 0s 31us/step - loss: 0.0527 - val_loss: 0.0544
Epoch 378/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0527 - val_loss: 0.0542
Epoch 379/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0527 - val_loss: 0.0545
Epoch 380/1000
5571/5571 [==============================] - 0s 34us/step - loss: 0.0527 - val_loss: 0.0543
Epoch 381/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0526 - val_loss: 0.0543
Epoch 382/1000
5571/5571 [==============================] - 0s 34us/step - loss: 0.0526 - val_loss: 0.0545
Epoch 383/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0526 - val_loss: 0.0542
Epoch 384/1000
5571/5571 [==============================] - 0s 34us/step - loss: 0.0526 - val_loss: 0.0542
Epoch 385/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0527 - val_loss: 0.0542
Epoch 386/1000
5571/5571 [==========================

5571/5571 [==============================] - 0s 33us/step - loss: 0.0520 - val_loss: 0.0536
Epoch 454/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0520 - val_loss: 0.0536
Epoch 455/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0520 - val_loss: 0.0535
Epoch 456/1000
5571/5571 [==============================] - 0s 31us/step - loss: 0.0520 - val_loss: 0.0539
Epoch 457/1000
5571/5571 [==============================] - 0s 30us/step - loss: 0.0520 - val_loss: 0.0536
Epoch 458/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0519 - val_loss: 0.0536
Epoch 459/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0519 - val_loss: 0.0537
Epoch 460/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0519 - val_loss: 0.0539
Epoch 461/1000
5571/5571 [==============================] - 0s 29us/step - loss: 0.0520 - val_loss: 0.0536
Epoch 462/1000
5571/5571 [==========================

In [121]:
scores= model.evaluate(val_x, val_y,verbose=1)
print(scores)

618/618 [==============================] - 0s 21us/step
0.05342113575069264


In [13]:
print(train_y )

[[[-0.13179678]]

 [[-0.13308433]]

 [[-0.10862081]]

 ...

 [[ 0.13086416]]

 [[-0.11377103]]

 [[ 0.03429764]]]
